In [1]:
import os
import cv2

def create_training_data(folder_path, classes, img_size):
    X_train = []
    y_train = []
    for category in classes:
        path = os.path.join(folder_path, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                X_train.append(new_array)
                y_train.append(class_num)
            except Exception as e:
                pass
    print("Converted image shape: " + str(X_train[0].shape))
    return(X_train, y_train)

In [2]:
import numpy as np

train_data_path = "D:/UQAC ETE 2023/8INF867_Fondamentaux de l'apprentissage automatique/Projet/svm/data"
classes = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]


X_train, y_train = create_training_data(train_data_path, classes, 48)

X_train = np.array(X_train)
y_train = np.array(y_train)

Converted image shape: (48, 48, 3)


In [ ]:
X_test = None
y_test = None
epochs = 2
batch_size = 256

number_of_labels = len(classes)
# Aplatir (vectoriser) les images
n_samples = len(X_train)
X_train = X_train.reshape((n_samples, -1))
input_shape = X_train.shape
print(input_shape)

REDUCTION DE DIMENSIONS

In [25]:
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV

pca = PCA(n_components = 10)
X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)

#explained_variance = pca.explained_variance_ratio_


CREATION DU MODELE SVM LINEAIRE

In [ ]:
from sklearn import svm

model = svm.LinearSVC(max_iter=60000)
# Création d'un classificateur calibré pour pouvoir tracer la courbe ROC_AUC
calibrated_svc = CalibratedClassifierCV(model, method='sigmoid')

ENTRAINEMENT

In [ ]:
from time import time
import datetime

start_time = time()

calibrated_svc.fit(X_train, y_train)

end_time = time()
duration = end_time-start_time

print("Training duration: ", str(datetime.timedelta(seconds=duration)))

C:\Users\harit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\harit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\harit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
support_vectors = model.support_vectors_
dec = model.decision_function([[1]])

In [ ]:
# from sklearn.metrics import classification_report
#
# print("TEST RESULTS")
# y_test_model = model.predict(X_test)
# print(classification_report(y_test, y_test_model))

y_pred = model.predict(X_test)
# Obtenir les probabilités de prédiction pour les données de test
y_pred_proba = calibrated_svc.predict_proba(X_test)

In [ ]:
# from joblib import dump
#
# dump(model, 'model_svm.joblib')

# later, the model can be retrieved with "clf = load('filename.joblib')"

In [ ]:
import metrics

metrics.compute_metrics(y_test, y_pred)
metrics.compute_roc_auc(y_test, y_pred_proba)
